In [3]:
import pandas as pd
from XBRL import lookup_cik
from SecEdgarDAO import SecEdgarDAO

In [9]:
class MorningstarDAO(object):

    def __init__(self):
        self.url = "http://financials.morningstar.com/ajax/ReportProcess4CSV.html?t=%s&reportType=%s&period=%d&dataType=A&order=asc&columnYear=5&number=%d"

        # reportType: is = Income Statement, cf = Cash Flow, bs = Balance Sheet
        self.reportType = {'bs': 'Balance Sheet', 'is': 'Income Statement', 'cf': 'Cash Flow'}

        # period: 12 for annual reporting, 3 for quarterly reporting
        self.period = 3

        # number: The units of the response data. 1 = None 2 = Thousands 3 = Millions 4 = Billions
        self.number = 2

    def load_fundamentals(self, symbol):
        f = pd.DataFrame()
        for report in self.reportType.keys():
            full_url = self.url % (symbol, report, self.period, self.number)
            df = pd.read_csv(full_url, header=1, index_col=0)
            df.index.name = ''
            if report == 'bs':
                df.index = df.index.where(~df.index.duplicated(keep='first'), 'Non-current ' + df.index)
                df.rename(index={'Deferred revenues': 'Current Deferred revenues'}, inplace=True)
                df.drop("Assets", axis=0, inplace=True, errors='ignore')
                df.drop("Current assets", axis=0, inplace=True, errors='ignore')
                df.drop("Cash", axis=0, inplace=True, errors='ignore')
                df.drop("Non-current assets", axis=0, inplace=True, errors='ignore')
                df.drop("Property, plant and equipment", axis=0, inplace=True, errors='ignore')
                df.drop("Liabilities and stockholders' equity", axis=0, inplace=True, errors='ignore')
                df.drop("Liabilities", axis=0, inplace=True, errors='ignore')
                df.drop("Current liabilities", axis=0, inplace=True, errors='ignore')
                df.drop("Non-current liabilities", axis=0, inplace=True, errors='ignore')
                df.drop("Stockholders' equity", axis=0, inplace=True, errors='ignore')
            if report == 'is':
                df.drop('TTM', axis=1, inplace=True, errors='ignore')
                df.index = df.index.where(~df.index.duplicated(keep='first'),
                                          'Weighted average shares outstanding ' + df.index)
                df.rename(index={'Basic': 'Earnings per share Basic', 'Diluted': 'Earnings per share Diluted'},
                          inplace=True)
                df.drop("Operating expenses", axis=0, inplace=True, errors='ignore')
                df.drop("Earnings per share", axis=0, inplace=True, errors='ignore')
                df.drop("Weighted average shares outstanding", axis=0, inplace=True, errors='ignore')
            if report == 'cf':
                df.drop('TTM', axis=1, inplace=True, errors='ignore')
                df.drop('Net income', axis=0, inplace=True, errors='ignore')
                df.drop("Cash Flows From Operating Activities", axis=0, inplace=True, errors='ignore')
                df.drop("Cash Flows From Investing Activities", axis=0, inplace=True, errors='ignore')
                df.drop("Cash Flows From Financing Activities", axis=0, inplace=True, errors='ignore')
                df.drop("Free Cash Flow", axis=0, inplace=True, errors='ignore')

            f = f.append(df)

        f.index = f.index.str.title()
        period_dates = (pd.to_datetime(f.columns) + pd.offsets.MonthEnd(0)).date

        # Filing date
        cik = lookup_cik(symbol)
        edgar = SecEdgarDAO()
        filing_dates = edgar.get_filings_dates(cik, period_dates)

        f.columns = filing_dates
        f.loc['Report Period'] = period_dates

        return f
symbol = 'AABA'
dao = MorningstarDAO()
df = dao.load_fundamentals(symbol)
df

INFO: 05/11/2018 09:03:02 AM Loading filings from https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001011006&type=10-%25&dateb=&owner=exclude&start=0&count=10&output=atom
INFO: 05/11/2018 09:03:03 AM Feed parsed.
INFO: 05/11/2018 09:03:03 AM Parsing data from yhoo-20170331.xml
INFO: 05/11/2018 09:03:04 AM Parsing data from yhoo-20161231.xml
INFO: 05/11/2018 09:03:04 AM Parsing data from yhoo-20160930.xml
INFO: 05/11/2018 09:03:04 AM Parsing data from yhoo-20160630.xml
INFO: 05/11/2018 09:03:05 AM Parsing data from yhoo-20160331.xml
INFO: 05/11/2018 09:03:05 AM Parsing data from yhoo-20151231.xml
INFO: 05/11/2018 09:03:05 AM Parsing data from yhoo-20150930.xml


,2016-05-10,2016-08-08,2016-11-09,2017-03-01,2017-05-09
,,,,,
Cash And Cash Equivalents,1.4796e+06,1.3254e+06,1.41131e+06,1.11947e+06,1.32891e+06
Restricted Cash,NaN,NaN,NaN,55168,NaN
Receivables,878135,991185,945659,1.15284e+06,940311
Investments,3.40442e+07,3.44124e+07,4.47692e+07,3.8107e+07,4.53111e+07
Property And Equipment,1.49651e+06,1.32624e+06,1.27333e+06,1.20994e+06,1.17772e+06
Goodwill,815979,431366,437609,415809,430463
Other Intangible Assets,316251,202116,181998,161644,142424
Other Assets,5.12759e+06,5.52554e+06,5.65257e+06,5.86118e+06,6.03777e+06
Total Assets,4.41582e+07,4.42143e+07,5.46716e+07,4.80831e+07,5.53687e+07


In [7]:
df.dropna()

,2016-05-10,2016-08-08,2016-11-09,2017-03-01,2017-05-09
,,,,,
Cash And Cash Equivalents,1.4796e+06,1.3254e+06,1.41131e+06,1.11947e+06,1.32891e+06
Receivables,878135,991185,945659,1.15284e+06,940311
Investments,3.40442e+07,3.44124e+07,4.47692e+07,3.8107e+07,4.53111e+07
Property And Equipment,1.49651e+06,1.32624e+06,1.27333e+06,1.20994e+06,1.17772e+06
Goodwill,815979,431366,437609,415809,430463
Other Intangible Assets,316251,202116,181998,161644,142424
Other Assets,5.12759e+06,5.52554e+06,5.65257e+06,5.86118e+06,6.03777e+06
Total Assets,4.41582e+07,4.42143e+07,5.46716e+07,4.80831e+07,5.53687e+07
Payables And Accrued Expenses,1.0184e+06,1.15448e+06,1.15245e+06,835842,1.06437e+06
